In [ ]:
#---#| default_exp spectral_library.decoy

# Decoy Libraries

In [ ]:
from alphabase.spectral_library.decoy import *

### Registered decoy methods

In [ ]:
decoy_lib_provider.decoy_dict

In [ ]:
#| hide
import os
import numpy as np
import pandas as pd

In [ ]:
#| hide
repeat = 3
peptides = ['AGHCEWQMK']*repeat
mods = ['Acetyl@Protein N-term;Carbamidomethyl@C;Oxidation@M']*repeat
sites = ['0;4;8']*repeat
peptides += ['AGHCEWQMKAADER']*repeat
mods += ['']*repeat
sites += ['']*repeat

precursor_df = pd.DataFrame({
    'sequence': peptides,
    'mods': mods,
    'mod_sites': sites
})
precursor_df['nAA'] = precursor_df['sequence'].str.len()
precursor_df['charge'] = 2
precursor_df

In [ ]:
#| hide
target_lib = SpecLibBase(['b_z1','b_z2','y_z1','y_z2'])
target_lib._precursor_df = precursor_df
target_lib.calc_precursor_mz()
target_lib._fragment_mz_df = pd.DataFrame()
target_lib._fragment_intensity_df = pd.DataFrame()
if not os.path.isdir('sandbox'):
    os.makedirs('sandbox')
target_lib.save_hdf('sandbox/decoy_lib.hdf')
target_lib.precursor_df

In [ ]:
decoy_lib = decoy_lib_provider.get_decoy_lib('diann', target_lib.copy())
decoy_lib.translate_to_decoy()

In [ ]:
decoy_lib.precursor_df

In [ ]:
#| hide
# call once with multiprocessing and once without
for mp_batch_size in [2, 10000]:

    decoy_lib = decoy_lib_provider.get_decoy_lib('pseudo_reverse', target_lib.copy())
    decoy_lib.translate_to_decoy(mp_batch_size=mp_batch_size)
    decoy_lib.calc_precursor_mz()
    assert np.allclose(decoy_lib.precursor_df.precursor_mz, target_lib.precursor_df.precursor_mz)

In [ ]:
decoy_lib.precursor_df

In [ ]:
#| hide
# call once with multiprocessing and once without
for mp_batch_size in [2, 10000]:
    decoy_lib = decoy_lib_provider.get_decoy_lib('diann', target_lib)
    decoy_lib.translate_to_decoy(mp_batch_size=mp_batch_size)
    if not os.path.isdir('sandbox'):
        os.makedirs('sandbox')
    decoy_lib.save_hdf('sandbox/decoy_lib.hdf')
    assert len(decoy_lib.precursor_df) > 0

In [ ]:
#| hide
speclib = SpecLibBase()
speclib.load_hdf('sandbox/decoy_lib.hdf')
assert len(speclib.precursor_df.values) > 0
assert len(speclib.fragment_mz_df.values) == 0

In [ ]:
#| hide
os.remove('sandbox/decoy_lib.hdf')

In [ ]:
#| hide
assert decoy_lib_provider.get_decoy_lib(None, target_lib) is None